# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2021-06-16 13:26 | by soge | Using ris library version: 2.2.1 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\HBKBQE

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')


Additional database connection details required:
User name (ris):soge
Password (ris)········


In [3]:
data =pd.read_excel('BQE Four Crossings Crash Data Request.xlsx')

### NODES Verification

In [4]:
y = len(data[data.columns[3::]][1:]) #matrix of all nodes without descriptive columns
l = list()

# separating all nodes from nan values
for i in range(y):
    
    for j in data[data.columns[3::]][1:].iloc[i]:
        
        
        if str(j) != 'nan':
            
             l.append(j)



In [5]:
nodes = tuple(map(int,l))

In [6]:
lion_nodes = db.dfquery(""" SELECT DISTINCT nodeid FROM node
                            WHERE nodeid in {} 
                            """.format(nodes))

In [7]:
# set of nodes not in node table
hbkbqe_nodes = set(nodes) - set(map(int,set(lion_nodes.nodeid))) 
hbkbqe_nodes

{15755, 90598, 90601}

### Updating nodeids 

In [8]:
# Removing incorrect nodes
ln = map(int,list(lion_nodes.nodeid))
ln.remove(90358)

# Appending correct nodes
ln.append(103538)
ln.append(103539)
ln.append(105928)
ln = tuple(ln)

### Reference Markers Verification

In [3]:
ref_mkr=pd.read_excel('BQE Four Crossings Crash Data Request.xlsx', sheet_name= 'Reference Marker')

In [4]:
markers = tuple(map(str,list(ref_mkr[ref_mkr.columns[6]][2:])))

In [6]:
print markers

('278IX2M23089', '278IX2M24089', '278IX2M23088', '278IX2M24088', '278IX2M23087', '278IX2M24087', '278IX2M23086', '278IX2M24086', '278IX2M23085', '278IX2M24085', '278IX2M23084', '278IX2M24084', '278IX2M23083', '278IX2M24083', '278IX2M23082', '278IX2M24082', '278IX2M23081', '278IX2M24081', '278IX2M23080', '278IX2M24080', '278IX2M23079', '278IX2M24079', '278IX2M23078', '278IX2M24078')


ris_markers = db.dfquery("""SELECT distinct l.segmentid,panel, l.geom
                            FROM public.cscl_referencemarker rm
                            JOIN lion l
                            on rm.segmentid::int = l.segmentid::int
                            WHERE panel in {}
                             """.format(markers))

rm_segs = tuple(map(int,list(ris_markers.segmentid)))

diff = set(ris_markers.panel) - set(markers)
diff

### NYSDOT Data

raw_crashes = db.dfquery("""SELECT ogc_fid, gid, case_num, case_yr, ref_mrkr, accd_dte, road_sys, 
                                   num_of_fat, num_of_inj, reportable, police_dep, intersect_, muni, 
                                   precinct, num_of_veh, accd_typ, locn, traf_cntl, light_cond, 
                                   weather, road_char, road_surf_, collision_, ped_loc, ped_actn, 
                                   ext_of_inj, regn_cnty_, accd_tme, rpt_agcy, dmv_accd_c, traf_way, 
                                   rdway_acc_, err_cde, comm_veh_a, highway_in, intersect1, utm_northi, 
                                   utm_eastin, crashid  
                            FROM nysdot_all
                            WHERE (nodeid::int in {n} or segmentid::int in {m})
                            and case_yr::int between 2016 and 2018
                            """.format(n=ln, m=rm_segs))

In [ ]:
raw_crashes = db.dfquery(""" SELECT ogc_fid, gid, case_num, case_yr, ref_mrkr, accd_dte, road_sys, 
                               num_of_fat, num_of_inj, reportable, police_dep, intersect_, muni, 
                               precinct, num_of_veh, accd_typ, locn, traf_cntl, light_cond, 
                               weather, road_char, road_surf_, collision_, ped_loc, ped_actn, 
                               ext_of_inj, regn_cnty_, accd_tme, rpt_agcy, dmv_accd_c, traf_way, 
                               rdway_acc_, err_cde, comm_veh_a, highway_in, intersect1, utm_northi, 
                               utm_eastin, crashid  
                            FROM nysdot_all
                            WHERE ref_mrkr in ('278IX2M23089', '278IX2M24089', '278IX2M23088', '278IX2M24088', '278IX2M23087', 
                                       '278IX2M24087', '278IX2M23086', '278IX2M24086', '278IX2M23085', '278IX2M24085', 
                                       '278IX2M23084', '278IX2M24084', '278IX2M23083', '278IX2M24083', '278IX2M23082', 
                                       '278IX2M24082', '278IX2M23081', '278IX2M24081', '278IX2M23080', '278IX2M24080', 
                                       '278IX2M23079', '278IX2M24079', '278IX2M23078', '278IX2M24078')
                            and case_yr::int between 2016 and 2018

                            """.format(n=ln, m=rm_segs))

In [15]:
#crashids
rc = tuple(list(raw_crashes.crashid))

In [16]:
raw_vehicle = db.dfquery("""SELECT *  
                            FROM nysdot_vehicle
                            WHERE crashid in {c}
                            and case_yr between 2016 and 2018
                            """.format(c=rc))



In [17]:
raw_contributing = db.dfquery("""SELECT *  
                                 FROM nysdot_appfactor
                                 WHERE crashid in {c}
                                 and case_yr between 2016 and 2018
                                 """.format(c=rc))



In [18]:
raw_crashes.to_csv('hbkbqe_raw_crashes.csv',index=False)

In [19]:
raw_vehicle.to_csv('hbkbqe_raw_vehicle.csv',index=False)

In [20]:
raw_contributing.to_csv('hbkbqe_raw_contributing.csv',index=False)